In [269]:
import xgboost
import pandas as pd
import sklearn.metrics as metr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.multioutput import MultiOutputRegressor

In [292]:
data = pd.read_csv('data/reformatted_data.csv').drop(['Unnamed: 0', 'REGION'], axis=1)
data = data[data.hh_500 > 50]

In [293]:
postamats = data[data.Postamat_daily == data.Postamat_daily].copy()
cashboxes = data[data.cashbox_daily == data.cashbox_daily].copy()

In [294]:
postamats.drop(['CITY', 'ADDRESS', 'cashbox_daily', 'cashbox_MAX', 'cashbox_trend', 'Postamat_trend', 'coords'], axis=1, inplace=True)

In [295]:
cashboxes.drop(['CITY', 'ADDRESS', 'Postamat_daily', 'Postamat_trend', 'cashbox_trend', 'coords'], axis=1, inplace=True)

In [296]:
print(postamats.isna().any().any())
print(cashboxes.isna().any().any())

False
False


In [297]:
cashboxes.columns

Index(['cashbox_daily', 'cashbox_MAX', 'hh_500', 'value1', 'value2',
       'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops'],
      dtype='object')

In [298]:
X_p = postamats[['hh_500', 'value1', 'value2', 'POPULATION',
       'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops']]
y_p = postamats[['Postamat_daily']]

In [299]:
X_c = cashboxes[['hh_500', 'value1', 'value2',
       'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops']]
y_c = cashboxes[['cashbox_daily', 'cashbox_MAX']]

In [300]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c)

In [301]:
xgb_p = xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4, learning_rate=0.1, subsample=0.85, gamma=0.1, n_estimators=500)
xgb_p.fit(X_train_p, y_train_p)
metr.mean_squared_error(xgb_p.predict(X_test_p), y_test_p)

1.767740775279459

In [302]:
xgb_c = MultiOutputRegressor(xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4,
                                                             learning_rate=0.1, subsample=0.85, 
                                                             gamma=0.1, n_estimators=500)).fit(X_train_c, y_train_c)
xgb_c.predict(X_test_c)
metr.mean_squared_error(xgb_c.predict(X_test_c), y_test_c)

7.565617579473066

Работает нормально - обучим на всех данных

In [303]:
xgb_p = xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4, learning_rate=0.1, subsample=0.85, gamma=0.1, n_estimators=500)
xgb_p.fit(X_p, y_p)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.85, verbosity=1)

In [304]:
xgb_c = MultiOutputRegressor(xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4,
                                                             learning_rate=0.1, subsample=0.85, 
                                                             gamma=0.1, n_estimators=500)).fit(X_c, y_c)

In [305]:
postamats_target = data[data.Postamat_daily != data.Postamat_daily].copy()
cashboxes_target = data[data.cashbox_daily != data.cashbox_daily].copy()

In [306]:
postamats_target.columns

Index(['ADDRESS', 'CITY', 'Postamat_daily', 'cashbox_daily', 'Postamat_trend',
       'cashbox_MAX', 'cashbox_trend', 'hh_500', 'value1', 'value2',
       'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'coords',
       'dist_to_closest_shop', 'dist_to_closest_postomat',
       'dist_to_competitor', 'dist_to_post', 'NEAR_shops', 'NEAR_stops'],
      dtype='object')

In [308]:
pred_p = xgb_p.predict(postamats_target.drop(['CITY', 'ADDRESS', 'Postamat_daily', 'cashbox_daily', 
                                              'Postamat_trend', 'cashbox_MAX', 'cashbox_trend', 'coords'], axis=1))
postamats_target['Postamat_daily'] = pred_p

In [309]:
pred_c = xgb_c.predict(cashboxes_target.drop(['CITY', 'ADDRESS', 'cashbox_daily', 'cashbox_MAX',
                                              'Postamat_daily', 'Postamat_trend', 'cashbox_trend', 'coords'], axis=1))

In [310]:
cashboxes_target['cashbox_daily'] = pred_c[:, 0]
cashboxes_target['cashbox_MAX'] = pred_c[:, 1]

In [311]:
postamats_target.to_csv('data/postamats_predicted.csv')

In [312]:
cashboxes_target.to_csv('data/cashboxes_predicted.csv')

In [343]:
postamats_target[postamats_target.Postamat_daily > 2.325].to_json('data/top50_postamats.json', orient='index')

In [330]:
cashboxes_target[cashboxes_target.cashbox_daily > 1.08].to_json('data/top100_cashboxes.json', orient='index')

In [342]:
postamats_target[postamats_target.Postamat_daily > 2.325].shape

(50, 23)

In [345]:
cashboxes_target[cashboxes_target.cashbox_daily > 1.5].shape

(46, 23)